Imagine you are working on a project that involves processing a large collection of text documents, such as research papers, legal documents, or customer service logs. Your task is to develop a system that can quickly retrieve the most relevant segments of text based on a user's query. Traditional keyword-based search methods might not be sufficient, as they often fail to capture the nuanced meanings and contexts within the documents. To address this challenge, you can use different types of retrievers based on LangChain.

Using retrievers is crucial for several reasons:

- Efficiency: Retrievers enable fast and efficient retrieval of relevant information from large datasets, saving time and computational resources.
- Accuracy: By leveraging advanced retrieval techniques, these tools can provide more accurate and contextually relevant results compared to traditional search methods.
- Versatility: Different retrievers can be tailored to specific use cases, making them adaptable to various types of text data and query requirements.
- Context awareness: Some retrievers, like the Parent Document Retriever, can consider the broader context of the document, enhancing the relevance of the retrieved segments.


We will learn about four types of retrievers: `Vector Store-backed Retriever`, `Multi-Query Retriever`, `Self-Querying Retriever`, and `Parent Document Retriever`. We will also learn the differences between these retrievers and understand the appropriate situations in which to use each one. By the end of this lab, you will be equipped with the skills to implement and utilize these retrievers in your projects.

In [ ]:
!pip install --user "chromadb==0.4.24" | tail -n 1


In [ ]:
pip install numpy==1.26.4


In [ ]:
pip install -U langchain-community

In [ ]:
#!pip install --user "lark==1.1.9" | tail -n 1
pip install -U lark


In [ ]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

The following functions are prerequisite knowledge for understanding the topic of this project—retrievers. These functions include:

- Building LLMs
- Splitting documents into chunks
- Building an embedding model

The relevant knowledge and details of these functions have been covered in previous lessons.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def llm():
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Lightweight version of Mixtral, openly available

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)

    # Use pipeline for easier generation
    text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    def generate(prompt):
        output = text_generator(
            prompt,
            max_new_tokens=256,
            temperature=0.5,
            do_sample=True,
            top_p=0.95,
            top_k=50
        )
        return output[0]["generated_text"]

    return generate


### Text Splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_splitter(data, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

#### Embedding model



In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def huggingface_embedding():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Fast, light, and effective

    embedding = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"}  # or "cuda" if GPU is available
    )
    return embedding


## Retrievers

A retriever is an interface designed to return documents based on an unstructured query. Unlike a vector store, which stores and retrieves documents, a retriever's primary function is to find and return relevant documents. While vector stores can serve as the backbone of a retriever, there are various other types of retrievers that can be used as well.

Retrievers take a string `query` as input and output a list of `Documents`.

### Vector Store-Backed Retriever

A vector store retriever is a type of retriever that utilizes a vector store to fetch documents. It acts as a lightweight wrapper around the vector store class, enabling it to conform to the retriever interface. This retriever leverages the search methods implemented by the vector store, such as similarity search and Maximum Marginal Relevance (MMR), to query texts stored within it.



In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt"

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("companypolicies.txt")
txt_data = loader.load()

In [ ]:
txt_data

Split `txt_data` into chunks. `chunk_size = 200`, `chunk_overlap = 20` has been set.


In [ ]:
chunks_txt = text_splitter(txt_data, 200, 20)

Store the embeddings into a `ChromaDB`.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Step 1: Define the embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}  # change to "cuda" if GPU available
)

# Step 2: Create Chroma vectorstore
vectordb = Chroma.from_documents(chunks_txt, embedding_model)


In [ ]:
chunks_txt

#### Simple similarity search

Here is an example of a simple similarity search based on the vector database.

For this demonstration, the query has been set to "email policy".


In [ ]:
query = "email policy"
retriever = vectordb.as_retriever()
docs = retriever.invoke(query)
docs

You can also specify `search kwargs` like `k` to limit the retrieval results.


In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 1})
docs = retriever.invoke(query)
docs

#### MMR retrieval

MMR in vector stores is a technique used to balance the relevance and diversity of retrieved results. It selects documents that are both highly relevant to the query and minimally similar to previously selected documents. This approach helps to avoid redundancy and ensures a more comprehensive coverage of different aspects of the query.

The following code is showing how to conduct an MMR search in a vector database. You just need to sepecify `search_type="mmr"`.


In [ ]:
retriever = vectordb.as_retriever(search_type="mmr")
docs = retriever.invoke(query)
docs

#### Similarity score threshold retrieval

You can also set a retrieval method that defines a similarity score threshold, returning only documents with a score above that threshold.


In [ ]:
retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.4}
)
docs = retriever.invoke(query)
docs

### Multi-Query Retriever

Distance-based vector database retrieval represents queries in high-dimensional space and finds similar embedded documents based on "distance". However, retrieval results may vary with subtle changes in query wording or if the embeddings do not accurately capture the data's semantics.

The `MultiQueryRetriever` addresses this by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and then takes the unique union of these results to form a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the `MultiQueryRetriever` can potentially overcome some limitations of distance-based retrieval, resulting in a richer and more diverse set of results.

A PDF document has been prepared to demonstrate this Multi-Query Retriever.


In [ ]:
pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf")
pdf_data = loader.load()
pdf_data[1]

Split document and store the embeddings into a vector database.


In [ ]:
vectordb.get()["ids"]

In [ ]:
# Split
chunks_pdf = text_splitter(pdf_data, 500, 20)

# VectorDB
ids = vectordb.get()["ids"]
vectordb.delete(ids=ids)  # ✅ pass as keyword argument
vectordb = Chroma.from_documents(documents=chunks_pdf, embedding=embedding_model)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

query = "What does the paper say about langchain?"

retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm()
)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

# Step 1: load model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 2: build pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.5,
    do_sample=True,
    top_p=0.95,
    top_k=50
)

# Step 3: wrap into LangChain LLM object
lc_llm = HuggingFacePipeline(pipeline=pipe)

# Now this works with MultiQueryRetriever:
retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=lc_llm
)

# Finally:
docs = retriever.invoke(query)


From the log results, you can see that the LLM generated three additional queries from different perspectives based on the given query.

The returned results are the union of the results from each query.


### Self-Querying Retriever

A Self-Querying Retriever, as the name suggests, has the ability to query itself. Specifically, given a natural language query, the retriever uses a query-constructing LLM chain to generate a structured query. It then applies this structured query to its underlying vector store. This enables the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documents but also to extract and apply filters based on the metadata of those documents.


In [ ]:
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from lark import lark

A couple of document pieces have been prepared where the `page_content` contains descriptions of movies, and the `meta_data` includes different attributes for each movie, such as `year`, `rating`, `genre`, and `director`. These attributes are crucial in the Self-Querying Retriever, as the LLM will use the metadata information to apply filters during the retrieval process.


In [ ]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

Now you can instantiate your retriever. To do this, you'll need to provide some upfront information about the metadata fields that your documents support, as well as a brief description of the document contents.


In [ ]:
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [ ]:
#Store the document's embeddings into a vector database.
vectordb = Chroma.from_documents(docs, embedding=embedding_model)

Use the `SelfQueryRetriever`.


In [ ]:
import lark

Now you can actually try using your retriever.


In [ ]:
# This example only specifies a filter
model_id = "Qwen/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 2: build pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.5,
    do_sample=True,
    top_p=0.95,
    top_k=50
)

# Step 3: wrap into LangChain LLM object
lc_llm = HuggingFacePipeline(pipeline=pipe)

document_content_description = "Brief summary of a movie."

retriever = SelfQueryRetriever.from_llm(
    lc_llm,
    vectordb,
    document_content_description,
    metadata_field_info,
)



In [ ]:
retriever.invoke("I want to watch a movie rated higher than 8.5")